<a href="https://colab.research.google.com/github/hekoh99/ai-security/blob/master/First%20Assignment/HaeunKo_ai_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline           #도표와 같은 그림, 소리, 애니메이션 과 같은 결과물들을 Rich output이라고 하는 데 이런 것들을 실행한 브라우저에서 바로 볼 수 있도록 해준다

20184128 고하은 ai-security 과제_1 코드들 옆에 # 주석달기


Transfer Learning Tutorial
==========================
**Author**: `Sasank Chilamkurthy <https://chsasank.github.io>`_

In this tutorial, you will learn how to train your network using
transfer learning. You can read more about the transfer learning at `cs231n
notes <http://cs231n.github.io/transfer-learning/>`__

Quoting these notes,

    In practice, very few people train an entire Convolutional Network
    from scratch (with random initialization), because it is relatively
    rare to have a dataset of sufficient size. Instead, it is common to
    pretrain a ConvNet on a very large dataset (e.g. ImageNet, which
    contains 1.2 million images with 1000 categories), and then use the
    ConvNet either as an initialization or a fixed feature extractor for
    the task of interest.

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.




In [0]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division # 파이썬 3처럼 print하고 나누기 연산을 하기 위한 것

import torch
import torch.nn as nn
import torch.optim as optim                            # 최적화 알고리즘의 구현체 제공
from torch.optim import lr_scheduler              # 미리 지정한 횟수의 epoch가 지날 때 마다 lr을 감소시켜줌 - 학습 초기에 빠르게 학습을 진행하다가 minimum에 다다를 것 같으면 lr을 줄여서 최적점을 더 잘 찾아가게
import numpy as np
import torchvision                                       #torchvision 데이터셋의 출력(output)은 [0,1] 범위를 갖는 PILImage이다.
from torchvision import datasets, models, transforms       #데이터셋들에 대한 데이터 로더와 이미지용 데이터 변환기를 불러옴
import matplotlib.pyplot as plt                           #사용환경 인터페이스를 제공 - 자동으로 figure와 axes를 생성하고 정의된 플롯을 얻을 수 있게 해줌
import time
import os
import copy

plt.ion()                                    # 실시간으로 형태가 변하는 그래프를 그릴 수 있도록 인터랙티브 그래프 그리기를 활성화 한다

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [0]:
# 학습을 위한 데이터 증가와 일반화 하기 + 그냥 검증을 위해서 일반화 하기
data_transforms = {                             #훈련 데이터와 검증 데이터를 담는 딕셔너리 자료형을 선언해준다
    'train': transforms.Compose([              #여러 개의 변환 함수를 하나로 조합해 묶어준다
        transforms.RandomResizedCrop(224),        #이미지를 랜덤한 크기 및 비율로 자른다. 여기선 224 사이즈로 자른다
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),                        #이미지를 torch.Tensor로 변환해준다
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    #tensor의 데이터 수치를 함수 안의 값들로 정규화 해준다(평균, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),                    #이미지를 지정한 크기(256)로 자른다
        transforms.CenterCrop(224),              #이미지의 중앙 부분을 크롭해서 [244,244] 크기로 만들어줌
        transforms.ToTensor(),                     #이미지를 또 tensor로 만들어준다
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    #마찬가지로 텐서의 데이터 정규화 과정
    ]),
}

data_dir = 'hymenoptera_data'                               #막시류 데이터 (벌과 개미를 분류하는 상황이므로)

path = {x: os.path.join(os.path.dirname(os.path.abspath(__file__)),data_dir,x)     #path['train']은 train set의 경로, path['val']은 val set의 경로
                for x in ['train', 'val']}                                        #윗 줄의 join함수는 문자열을 이어 붙여주는 함수

image_datasets = {x: datasets.ImageFolder(path[x],
                                          data_transforms[x]),
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,     #torch.utils.data.DataLoader를 이용해서 입력 데이터를 불러온다, 배치 사이즈를 4로
                                             shuffle=True, num_workers=4)             #num_workers는 스레드 갯수를 설정하는 것이다.
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}         #dataset_sizes['train'] = train set 사진 개수를 의미, dataset_sizes['val']은 val set 사진 수
class_names = image_datasets['train'].classes                               

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")            #GPU를 이용할 수 있는지를 확인해본다

Visualize a few images
^^^^^^^^^^^^^^^^^^^^^^
Let's visualize a few training images so as to understand the data
augmentations.



In [0]:
def imshow(inp, title=None):                           #이미지를 보여주는 함수 선언
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean                      #정규화를 해제함
    inp = np.clip(inp, 0, 1)                   # np.clip(배열, 최소값 기준, 최대값 기준)을 사용하면 해당 최소값과 최대값 기준으로 이 범위를 벗어나는 값에 대해 일괄적으로 최소값, 최대값으로 대치해줌.
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)                        # 이미지들이 업로드 될 때 까지 잠시 기다려주는 작업


inputs, classes = next(iter(dataloaders['train']))   #한개의 배치 사이즈 만큼 이미지를 불러온다. 위에서 배치 사이즈를 4로 했으므로 사진이 4장 로드된다

out = torchvision.utils.make_grid(inputs)       #로드된 데이터에 make_grid 함수를 통해 그리드를 추가한다

imshow(out, title=[class_names[x] for x in classes])        #이미지를 출력한다

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):      #학습을 하기 위해 학습모델 함수 선언
    since = time.time()                                         #총 소요시간을 계산 하기 위해 시작시간을 기록한다

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):                                  #데이터 셋을 수차례 반복한다 (num_epochs만큼 학습 세대를 반복하는 것)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))         #epoch를 카운트
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:                           #훈련과 검증 순으로 진행 => 학습과 검사를 하나의 코드로 합쳐놓은 형식
            if phase == 'train':
                scheduler.step()
                model.train()                                      # 모델을 훈련 모드로 설정해준다
            else:
                model.eval()                                       # 모델을 평가 모드로 설정

            running_loss = 0.0                                      #손실함수의 결과값 즉 오차 loss를 최소로 하는 것이 학습의 목표. 초기값을 0으로 설정해두고 밑에 학습을 진행하면서 갱신해준다. 
            running_corrects = 0
                                                                    #데이터 반복 과정
            for inputs, labels in dataloaders[phase]:             #dataloader로부터 dataset과 그에 해당하는 label을 불러온다
                inputs = inputs.to(device)                         #GPU로 입력 데이터를 올린다
                labels = labels.to(device)                        #GPU로 라벨을 올린다

               
                optimizer.zero_grad()                                #변화도(Grandient) 매개변수를 0으로 초기화 한다

                # forward
                with torch.set_grad_enabled(phase == 'train'):                       # 학습 시에만 연산 기록을 추적하게 한다
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)                     #마지막 layer에서 가장 값이 큰 1개의 class를 예측 값으로 지정함
                    loss = criterion(outputs, labels)

                    if phase == 'train':                                #training 모드에서는 weight를 갱신해준다 (역전파 + 최적화)
                        loss.backward()
                        optimizer.step()                               #매개 변수를 갱신해주는 함수

                running_loss += loss.item() * inputs.size(0)                     #통계 출력, loss 갱신해준다
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())                    # 레퍼런스 대입시의 문제를 해결하기 위해 copy모듈의 deepcopy를 사용해 복사. 

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)                   #가장 적합한 모델 가중치를 반환
    return model

Visualizing the model predictions
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Generic function to display predictions for a few images




In [0]:
def visualize_model(model, num_images=6):                         # 일부 이미지에 대한 예측값을 보여주는 일반화 된 함수 선언해줌. 
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()                                   # figure 함수를 불러냄

    with torch.no_grad():                                                 # 트래킹 히스토리를 막기 위해 torch.no_grad 코드 블럭으로 묶음
        for i, (inputs, labels) in enumerate(dataloaders['val']):            # 검증 데이터에 있을 때  
            inputs = inputs.to(device)                              # 모델에서 사용하는 input Tensor들은 input = input.to(device) 을 호출해야함
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')                                                 # 가로 세로 축 프레임을 없앤다 (plt를 캔버스 삼아 그림을 나타내려면 축이 없어야 함)
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [0]:
model_ft = models.resnet18(pretrained=True)        #pretrained model을 가져온다. 여기서는 ResNet18(오류율이 굉장히 적은 CNN 모델)이라는 모델을 가져왔다
num_ftrs = model_ft.fc.in_features                 #model_ft.fc.in_features는 ResNet18모델의 마지막 단계에서 출력 노드의 갯수를 구해주는 함수다
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)
                                         #손실 함수와 optimizer 정의하기
criterion = nn.CrossEntropyLoss()       #손실 함수로는 crossEntropy를 사용하고 밑의 optimizer로는 SGD(Stochastic Gradient Descent)를 사용했다

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)         #모든 매개 변수가 최적화 되었는지 알아본다
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)      #학습률(lr)이 일곱 epoch마다 0.1씩 감소한다

Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [0]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,         # 모델을 훈련시키며 training accuracy와 validataion accuracy를 관찰
                       num_epochs=25)

In [0]:
visualize_model(model_ft)

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [0]:
# 마지막 계층을 제외한 신경망의 모든 부분을 고정해서 backward()가 실행되는 중에 경사도가 계산되지 않도록 한다
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# 새로 생성된 모듈의 매개변수는 기본값이 requires_grad = True이다
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# 전과는 다르게 마지막 계층의 매개 변수들만 최적화 되는지 본다(이전엔 모든 매개변수들이 최적화 되는지 확인했다)
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)    # 학습률(lr) 7 epoch 마다 0.1씩 감소

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [0]:
model_conv = train_model(model_conv, criterion, optimizer_conv,                    
                         exp_lr_scheduler, num_epochs=25)

In [0]:
visualize_model(model_conv)

plt.ioff()                                 #즉각적으로 이미지를 그리는 기능을 끄는 함수. 그리는 시간이 오래걸리는 이미지 작업의 경우 이렇게 하면 시간을 더 절약할 수 있다
plt.show()                                  #이미지 띄우는 함수

출처 : https://pytorch.org/tutorials/index.html?highlight=machine%20learning